In [2]:
import requests
from bs4 import BeautifulSoup

# Define the URL for the SEC EDGAR search page
url = "https://www.sec.gov/files/company_tickers_exchange.json"

# Fetch the HTML content of the search page
response = requests.get(url)
html_content = response.content

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find the link to the JSON file (this is an example, actual implementation may vary)
json_link = None
for link in soup.find_all('a'):
    href = link.get('href')
    if href and 'json' in href:
        json_link = href
        break

# If a JSON link is found, fetch its content
if json_link:
    json_url = f"https://data.sec.gov/api/xbrl/companyconcept/CIK0001652044/us-gaap/AccountsPayableCurrent.json"
    json_response = requests.get(json_url)
    json_content = json_response.json()
    print(json_content)
else:
    print("No JSON link found.")

No JSON link found.
